# Testing on PISA dataset with randomized missing data, computing counterexamples relative to the amount of non-missing values of an item

1. Importing package and dataset

In [1]:
!pip install iita_python
!git clone https://gist.github.com/717f0147675b0c8ed25e50d583c943bf.git

import numpy as np
import iita_python as iita
import iita_python.fit_metrics as iita_fm
from iita_python.utils import read_rp
from random import randint, choice, shuffle

Cloning into '717f0147675b0c8ed25e50d583c943bf'...
remote: Enumerating objects: 3, done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 3 (from 1)
Receiving objects: 100% (3/3), done.


2. Testing function

In [2]:
def test(metric, skips):
  correct = True
  correct_qo = None
  correct_count = 0
  data = read_rp('./717f0147675b0c8ed25e50d583c943bf/pisa.csv')

  while (correct and correct_count < data.shape[0]*data.shape[1] - 10):
    print(correct_count)
    test_dataset = iita.Dataset(data)
    unfolded_ce = iita.unfold_examples(test_dataset.relative_ce)
    qos = iita.ind_gen(unfolded_ce, test_dataset.items)

    best_qo_id = -1
    best_qo_diff = float('inf')
    for i, qo in enumerate(qos):
      qo_diff = metric(test_dataset, qo)
      if (qo_diff < best_qo_diff):
        best_qo_diff = qo_diff
        best_qo_id = i

    best_qo = sorted([(int(a), int(b)) for a, b in qos[best_qo_id].get_edge_list()])
    if (correct_qo is None):
      correct_qo = best_qo

    if (best_qo != correct_qo):
      correct = False
    else:
      correct_count += skips

      for _ in range(skips):
        while (True):
          a = randint(0, test_dataset.items - 1)
          b = randint(0, test_dataset.subjects - 1)
          # print(f'trying {a} {b}')
          # print(data.loc[b, a])
          if (not (np.isnan(data.loc[b, a]) or (np.nansum(data.to_numpy(), axis=0)[a] == 1))):
            break;
        data.loc[b, a] = np.nan

  return correct_count

In [3]:
def iter_test(metric, skips, iters, testf, **kwargs):
  iter_res = []
  for i in range(iters):
    print(f'ITER {i}')

    res = testf(metric, skips, **kwargs)
    print(res)
    iter_res.append(res)
  return iter_res

3. Running the tests

In [ ]:
iters = 100 #amount of iterations to do
skips = 5 #amount of missing values to add at a time

res = iter_test(iita_fm.mini_iita_fit, skips, iters, test)

4. Analyzing the tests

Running the tests is quite a long process, so I did it for 100 iterations and saved the results in the next cell

Skip the next cell if you run the tests on your own

In [4]:
res = [
    530, 440, 380, 620, 330, 620, 390, 370, 620, 550,
    540, 510, 390, 690, 390, 570, 460, 460, 300, 630,
    440, 400, 530, 610, 310, 270, 420, 600, 460, 450,
    390, 640, 410, 580, 140, 570, 380, 180, 440, 590,
    380, 690, 520, 140, 440, 360, 310, 230, 550, 460,
    460, 560, 450, 370, 470, 530, 670, 360, 570, 550,
    450, 560, 560, 660, 250, 250, 80, 550, 550, 450,
    260, 520, 360, 330, 680, 600, 540, 440, 450, 580,
    450, 330, 570, 620, 540, 510, 190, 400, 410, 420,
    630, 580, 330, 400, 420, 580, 460, 280, 630, 540
]

In [5]:
data = read_rp('./717f0147675b0c8ed25e50d583c943bf/pisa.csv')
res = (np.array(res) / (data.shape[0] * data.shape[1])).round(3)

4.1. Average

In [6]:
np.mean(res).round(3)

np.float64(0.271)

4.2. Standard deviation

In [7]:
np.std(res).round(3)

np.float64(0.078)

5. Testing function for biased item choice

In [8]:
def test_biased_items(metric, skips, bias):
  correct = True
  correct_qo = None
  correct_count = 0
  data = read_rp('./717f0147675b0c8ed25e50d583c943bf/pisa.csv')

  choicePool = []
  items = list(range(data.shape[1]))
  shuffle(items)

  for i, item in enumerate(items):
    for _ in range(bias[i]):
      choicePool.append(item)

  while (correct and correct_count < data.shape[0]*data.shape[1] - 10):
    print(correct_count)
    test_dataset = iita.Dataset(data)
    unfolded_ce = iita.unfold_examples(test_dataset.relative_ce)
    qos = iita.ind_gen(unfolded_ce, test_dataset.items)

    best_qo_id = -1
    best_qo_diff = float('inf')
    for i, qo in enumerate(qos):
      qo_diff = metric(test_dataset, qo)
      if (qo_diff < best_qo_diff):
        best_qo_diff = qo_diff
        best_qo_id = i

    best_qo = sorted([(int(a), int(b)) for a, b in qos[best_qo_id].get_edge_list()])
    if (correct_qo is None):
      correct_qo = best_qo

    if (best_qo != correct_qo):
      correct = False
    else:
      correct_count += skips

      for _ in range(skips):
        while (True):
          a = choice(choicePool)
          b = randint(0, test_dataset.subjects - 1)
          # print(f'trying {a} {b}')
          # print(data.loc[b, a])
          if (not (np.isnan(data.loc[b, a]) or (np.nansum(data.to_numpy(), axis=0)[a] == 1))):
            break;
        data.loc[b, a] = np.nan

  return correct_count

In [ ]:
iters = 40 # amount of iterations to do
skips = 5 # amount of missing values to add at a time
bias = [15, 11, 5, 3, 2] # probability distribution across the items. higher = more often

res_biased = iter_test(iita_fm.mini_iita_fit, skips, iters, test_biased_items, bias=bias)

Running the tests is quite a long process, so I did it for 40 iterations and saved the results in the next cell

Skip the next cell if you run the tests on your own

In [9]:
res_biased = np.array([320, 115, 170, 270, 225, 140, 365, 225, 235, 140,
    205, 275, 225, 165, 210, 235, 240, 420, 165, 185,
    190, 190, 215, 180, 145, 240, 275, 275, 210, 255,
    490, 260, 140, 215, 185, 125, 325, 405, 190, 150
])

In [10]:
res_biased = (np.array(res_biased) / (data.shape[0] * data.shape[1])).round(3)

In [11]:
np.mean(res_biased).round(3)

np.float64(0.135)

In [12]:
np.std(res_biased).round(3)

np.float64(0.048)